<a href="https://colab.research.google.com/github/mbendjilali/Archive/blob/main/Classification_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Adresse mail : michelle.aubrun@thalesaleniaspace.com à rendre le 1 avril 2022

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import copy

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

##Neural Network *model*

In [13]:
class MyNetwork2(nn.Module):

    def __init__(self, num_out):
        self.num_out = num_out # output size of the last layer
        super(MyNetwork2, self).__init__()

        # Load and tune resnet18
        self.model = torchvision.models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_out)
        self.model = self.model.to(device)

    # Forward function of the classification model
    def forward(self, input):
        output = self.model.forward(input)
        return output

##Load data

In [19]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainingset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                           download=True, transform=transform)

lengths = [int(len(trainingset)*0.9), int(len(trainingset)*0.1)]
trainset, valset = torch.utils.data.random_split(trainingset, lengths)

train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)
val_dataloader = torch.utils.data.DataLoader(valset, batch_size=32, 
                                        shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
loader_test = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


##Training the model

In [27]:
# --------------- Model architecture -----------------

num_epochs = 10
best_acc = 0.0
n = 10 # number of classes

# define model architecture
network = MyNetwork2(n)


# define loss function
criterion = nn.CrossEntropyLoss()

# define optimizer
optimizer = torch.optim.Adam([dict(params=network.parameters(), lr = 0.00008),])


# Training
for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        epoch_loss = []
        epoch_acc = 0

        if phase=='train':
            # Model in training mod

            loader = train_dataloader
        else:
            # Model in evaluation mod
            loader = val_dataloader

        for i, data in enumerate(loader):
            # Load image and class inside a batch 
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Set the gradient to zero
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'): # compute or not the gradient
                # Apply the model & compute the loss
                outputs = network(inputs)
                loss = criterion(outputs, labels)
                preds = torch.argmax(outputs, 1)

                epoch_acc += torch.sum(preds == labels)
                epoch_loss.append(loss.item())

                if phase == 'train':
                    # compute the gradient & update parameters
                    loss.backward()
                    optimizer.step()
        # Display loss
        mean_loss_epoch = np.mean(epoch_loss)
        std_loss_epoch = np.std(epoch_loss)
        if phase == 'train':
            print(f'Epoch: {epoch + 1}, Loss_train: {mean_loss_epoch:.4f} (± {std_loss_epoch:.4f})')
        else:
            print(f'Epoch: {epoch + 1}, Loss_val: {mean_loss_epoch:.4f} (± {std_loss_epoch:.4f})')

            if epoch_acc> best_acc :
                # Keep curent model with copy function
                best_current_model = copy.deepcopy(network.state_dict())

# Save best classification model
torch.save(best_current_model, 'bestClassificationModel.pt')

Epoch: 1, Loss_train: 1.5732 (± 0.2721)
Epoch: 1, Loss_val: 1.3810 (± 0.2055)
Epoch: 2, Loss_train: 1.1988 (± 0.2014)
Epoch: 2, Loss_val: 1.2269 (± 0.1981)
Epoch: 3, Loss_train: 0.9877 (± 0.1874)
Epoch: 3, Loss_val: 1.1291 (± 0.1920)
Epoch: 4, Loss_train: 0.8205 (± 0.1819)
Epoch: 4, Loss_val: 1.1133 (± 0.2649)
Epoch: 5, Loss_train: 0.6578 (± 0.1774)
Epoch: 5, Loss_val: 1.1480 (± 0.2927)
Epoch: 6, Loss_train: 0.5185 (± 0.1600)
Epoch: 6, Loss_val: 1.1794 (± 0.3314)
Epoch: 7, Loss_train: 0.4001 (± 0.1459)
Epoch: 7, Loss_val: 1.2806 (± 0.3568)
Epoch: 8, Loss_train: 0.3130 (± 0.1308)
Epoch: 8, Loss_val: 1.4060 (± 0.3812)
Epoch: 9, Loss_train: 0.2533 (± 0.1235)
Epoch: 9, Loss_val: 1.4783 (± 0.4387)
Epoch: 10, Loss_train: 0.2150 (± 0.1140)
Epoch: 10, Loss_val: 1.5370 (± 0.4085)


##Evaluate the model

In [32]:
# Load classification model with its weights
bestNetwork = MyNetwork2(10)

bestNetwork.load_state_dict(torch.load('bestClassificationModel.pt'))

# Generate batches of test data 

preds_all = []
labels_all = []
correct = 0
total = 0
# Testing
with torch.no_grad(): # no gradient
    for i, data in enumerate(loader_test):
        # Load image and class inside a batch 
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels_all.extend(labels)
        total += labels.size(0)

        # Apply the model
        outputs = bestNetwork(inputs)
        preds = torch.argmax(outputs, 1)
        preds_all.append(preds)
        correct += (preds == labels).sum().item()



# Display results
print(total)
print(correct)
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

10000
6362
Accuracy of the network on the 10000 test images: 63 %
